In [1]:
import keras
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches
import seaborn as sns
from tqdm import tqdm
%matplotlib inline

/home/paperspace/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 1. Read Data

In [17]:
# read saved
data2 = pd.read_csv("data_train_concat.csv", index_col=False)

/home/paperspace/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
Y = data2['x_exit_19'].between(3750901.5068, 3770901.5068) \
    & data2['y_exit_19'].between((-19268905.6133), -19208905.6133)

In [19]:
X = data2.drop([x+str(y) for y in range(20) for x in ['Unnamed: 0_', 'hash_', 'trajectory_id_', 'time_entry_', 'time_exit_']] \
    + ['x_exit_19', 'y_exit_19','vmax_19', 'vmin_19', 'vmean_19'] + ['Unnamed: 0'], axis=1)

In [20]:
# extrapolation
for i in range(18,-1,-1):
    cur = str(i)
    nex = str(i+1)
    nan = X['x_entry_'+cur].isnull()
    for x,y in zip(['x_entry_'+cur, 'y_entry_'+cur, 'entry_hour_'+cur, 'entry_minute_'+cur, 'entry_second_'+cur],
                   ['x_entry_'+nex, 'y_entry_'+nex, 'entry_hour_'+nex, 'entry_minute_'+nex, 'entry_second_'+nex]):
        X.iloc[np.where(nan)[0], X.columns.get_loc(x)] = X[y][nan]
    for x,y in zip(['x_exit_'+cur, 'y_exit_'+cur, 'exit_hour_'+cur, 'exit_minute_'+cur, 'exit_second_'+cur],
                   ['x_entry_'+nex, 'y_entry_'+nex, 'entry_hour_'+nex, 'entry_minute_'+nex, 'entry_second_'+nex]):
        X.iloc[np.where(nan)[0], X.columns.get_loc(x)] = X[y][nan]

In [21]:
for i in range(19):
    X['distance_'+str(i)] = ((X['x_entry_'+str(i)]-X['x_exit_'+str(i)]).pow(2)+(X['y_entry_'+str(i)]-X['y_exit_'+str(i)]).pow(2)).pow(0.5)
    X['speed_'+str(i)] = X['distance_'+str(i)]/X['duration_'+str(i)]
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(0)

In [44]:
# rearrange for CNN
X['x_exit_19'] = X['x_entry_19']
X['y_exit_19'] = X['y_entry_19']
X['distance_19'] = 0
X['speed_19'] = 0
X['vmin_19'] = 0
X['vmean_19'] = 0
X['vmax_19'] = 0
sortcol = sorted(X.columns.tolist(), key=lambda x: (int(x[x.rfind("_")+1:]), x[:x.rfind("_")]))
X = X[sortcol]

In [186]:
X.columns.tolist()

['distance_0',
 'duration_0',
 'entry_hour_0',
 'entry_minute_0',
 'entry_second_0',
 'exit_hour_0',
 'exit_minute_0',
 'exit_second_0',
 'speed_0',
 'vmax_0',
 'vmean_0',
 'vmin_0',
 'x_entry_0',
 'x_exit_0',
 'y_entry_0',
 'y_exit_0',
 'distance_1',
 'duration_1',
 'entry_hour_1',
 'entry_minute_1',
 'entry_second_1',
 'exit_hour_1',
 'exit_minute_1',
 'exit_second_1',
 'speed_1',
 'vmax_1',
 'vmean_1',
 'vmin_1',
 'x_entry_1',
 'x_exit_1',
 'y_entry_1',
 'y_exit_1',
 'distance_2',
 'duration_2',
 'entry_hour_2',
 'entry_minute_2',
 'entry_second_2',
 'exit_hour_2',
 'exit_minute_2',
 'exit_second_2',
 'speed_2',
 'vmax_2',
 'vmean_2',
 'vmin_2',
 'x_entry_2',
 'x_exit_2',
 'y_entry_2',
 'y_exit_2',
 'distance_3',
 'duration_3',
 'entry_hour_3',
 'entry_minute_3',
 'entry_second_3',
 'exit_hour_3',
 'exit_minute_3',
 'exit_second_3',
 'speed_3',
 'vmax_3',
 'vmean_3',
 'vmin_3',
 'x_entry_3',
 'x_exit_3',
 'y_entry_3',
 'y_exit_3',
 'distance_4',
 'duration_4',
 'entry_hour_4',
 'ent

In [51]:
from sklearn.preprocessing import StandardScaler, scale

In [52]:
scaler = StandardScaler()

In [53]:
X[:] = scaler.fit_transform(X)

In [54]:
X[:5]

,distance_0,duration_0,entry_hour_0,entry_minute_0,entry_second_0,exit_hour_0,exit_minute_0,exit_second_0,speed_0,vmax_0,...,exit_minute_19,exit_second_19,speed_19,vmax_19,vmean_19,vmin_19,x_entry_19,x_exit_19,y_entry_19,y_exit_19
0,-0.014382,-0.019108,0.068311,-1.408640,0.405377,0.068284,-1.408663,0.405325,-0.009816,-0.010304,...,-0.409425,0.482560,0.0,0.0,0.0,0.0,-1.697862,-1.697862,-0.785789,-0.785789
1,-0.014382,-0.019108,1.816504,0.025205,-0.065709,1.816494,0.025155,-0.065751,-0.009816,-0.010304,...,0.237699,1.368141,0.0,0.0,0.0,0.0,-1.244502,-1.244502,-0.578554,-0.578554
2,-0.014382,-0.019108,1.067279,0.484035,-1.066766,1.067261,0.483977,-1.066786,-0.009816,-0.010304,...,-0.350596,1.545257,0.0,0.0,0.0,0.0,-0.188570,-0.188570,-2.021392,-2.021392
3,-0.014382,-0.019108,0.567795,1.573758,-0.948995,0.567773,1.573679,-0.949017,-0.009816,-0.010304,...,0.178869,1.722373,0.0,0.0,0.0,0.0,0.810236,0.810236,0.577988,0.577988
4,-0.014382,-0.019108,-1.679881,-1.638055,-1.184537,-1.679926,-1.638074,-1.184555,-0.009816,-0.010304,...,-0.997720,-1.170525,0.0,0.0,0.0,0.0,-1.402849,-1.402849,-0.065226,-0.065226


## 1.4 Train/Test Split

In [55]:
from sklearn.model_selection import train_test_split

In [56]:
X[:5]

,distance_0,duration_0,entry_hour_0,entry_minute_0,entry_second_0,exit_hour_0,exit_minute_0,exit_second_0,speed_0,vmax_0,...,exit_minute_19,exit_second_19,speed_19,vmax_19,vmean_19,vmin_19,x_entry_19,x_exit_19,y_entry_19,y_exit_19
0,-0.014382,-0.019108,0.068311,-1.408640,0.405377,0.068284,-1.408663,0.405325,-0.009816,-0.010304,...,-0.409425,0.482560,0.0,0.0,0.0,0.0,-1.697862,-1.697862,-0.785789,-0.785789
1,-0.014382,-0.019108,1.816504,0.025205,-0.065709,1.816494,0.025155,-0.065751,-0.009816,-0.010304,...,0.237699,1.368141,0.0,0.0,0.0,0.0,-1.244502,-1.244502,-0.578554,-0.578554
2,-0.014382,-0.019108,1.067279,0.484035,-1.066766,1.067261,0.483977,-1.066786,-0.009816,-0.010304,...,-0.350596,1.545257,0.0,0.0,0.0,0.0,-0.188570,-0.188570,-2.021392,-2.021392
3,-0.014382,-0.019108,0.567795,1.573758,-0.948995,0.567773,1.573679,-0.949017,-0.009816,-0.010304,...,0.178869,1.722373,0.0,0.0,0.0,0.0,0.810236,0.810236,0.577988,0.577988
4,-0.014382,-0.019108,-1.679881,-1.638055,-1.184537,-1.679926,-1.638074,-1.184555,-0.009816,-0.010304,...,-0.997720,-1.170525,0.0,0.0,0.0,0.0,-1.402849,-1.402849,-0.065226,-0.065226


In [121]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2, random_state=42)

In [122]:
(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

((107250, 320), (26813, 320), (107250,), (26813,))

# 2. Model Building

In [80]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Activation, Conv1D, Flatten, MaxPooling1D
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD

In [126]:
train_x = np.expand_dims(train_x, axis=2)

In [134]:
test_x = np.expand_dims(test_x, axis=2)

In [127]:
train_x.shape

(107250, 320, 1)

In [182]:
# single head CNN
model = Sequential()
model.add(Conv1D(100, 16*3, strides=16, input_shape=(320,1)))
model.add(Activation('relu'))
model.add(Conv1D(100, 2, strides=1))
model.add(Activation('relu'))
# model.add(Conv1D(100, 16*2, strides=16))
# model.add(Activation('relu'))
# model.add(MaxPooling1D(pool_size=48))
model.add(Flatten())
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dense(500))
model.add(Activation('relu'))
# model.add(Dense(500))
# model.add(Activation('sigmoid'))
model.add(Dense(1, activation='sigmoid'))
# model.add(Dense(2))

In [183]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_61 (Conv1D)           (None, 18, 100)           4900      
_________________________________________________________________
activation_88 (Activation)   (None, 18, 100)           0         
_________________________________________________________________
conv1d_62 (Conv1D)           (None, 17, 100)           20100     
_________________________________________________________________
activation_89 (Activation)   (None, 17, 100)           0         
_________________________________________________________________
flatten_34 (Flatten)         (None, 1700)              0         
_________________________________________________________________
dense_66 (Dense)             (None, 500)               850500    
_________________________________________________________________
activation_90 (Activation)   (None, 500)               0         
__________

In [184]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
# model.compile(optimizer='rmsprop',
#               loss='mse',
#               metrics=['mse', 'mae'])

In [185]:
callbacks = [EarlyStopping(monitor='val_loss', patience=10, verbose=2)]
model.fit(train_x, train_y, epochs=100, batch_size=200, 
          validation_data=(test_x, test_y), callbacks = callbacks)

Train on 107250 samples, validate on 26813 samples
Epoch 1/100
107250/107250 [==============================] - 6s 58us/step - loss: 0.2595 - acc: 0.8886 - val_loss: 0.1935 - val_acc: 0.9239
Epoch 2/100
107250/107250 [==============================] - 5s 50us/step - loss: 0.1849 - acc: 0.9232 - val_loss: 0.1842 - val_acc: 0.9257
Epoch 3/100
107250/107250 [==============================] - 5s 49us/step - loss: 0.1742 - acc: 0.9278 - val_loss: 0.1737 - val_acc: 0.9290
Epoch 4/100
107250/107250 [==============================] - 5s 50us/step - loss: 0.1674 - acc: 0.9303 - val_loss: 0.1736 - val_acc: 0.9307
Epoch 5/100
107250/107250 [==============================] - 5s 49us/step - loss: 0.1620 - acc: 0.9326 - val_loss: 0.1729 - val_acc: 0.9323
Epoch 6/100
107250/107250 [==============================] - 5s 49us/step - loss: 0.1567 - acc: 0.9346 - val_loss: 0.1723 - val_acc: 0.9306
Epoch 7/100
107250/107250 [==============================] - 5s 49us/step - loss: 0.1519 - acc: 0.9373 - val_

In [191]:
model.evaluate(test_x, test_y)

26813/26813 [==============================] - 1s 43us/step


[0.22181390974390972, 0.9116473352722803]

In [17]:
# 351 rounds (0.983474)
models = xgb.XGBClassifier(max_depth=10, learning_rate=0.03, n_estimators=20000, n_jobs=-1,
                           subsample=0.8, colsample_bytree=0.8, seed=420)
#
# models = xgb.XGBClassifier(max_depth=6, learning_rate=0.05, n_estimators=20000,
#                            subsample=0.8, colsample_bytree=0.9, reg_alpha=65, seed=420, n_jobs=-1)

In [18]:
models.fit(train_x, train_y, eval_metric='auc', early_stopping_rounds=100,
           eval_set=[(train_x, train_y), (test_x, test_y)])


[0]	validation_0-auc:0.982631	validation_1-auc:0.976437
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.984759	validation_1-auc:0.978174
[2]	validation_0-auc:0.985666	validation_1-auc:0.97841
[3]	validation_0-auc:0.986481	validation_1-auc:0.979201
[4]	validation_0-auc:0.986054	validation_1-auc:0.978296
[5]	validation_0-auc:0.985754	validation_1-auc:0.977627
[6]	validation_0-auc:0.985837	validation_1-auc:0.976427
[7]	validation_0-auc:0.985561	validation_1-auc:0.975969
[8]	validation_0-auc:0.985545	validation_1-auc:0.975795
[9]	validation_0-auc:0.986496	validation_1-auc:0.976779
[10]	validation_0-auc:0.986844	validation_1-auc:0.977142
[11]	validation_0-auc:0.98719	validation_1-auc:0.977489
[12]	validation_0-auc:0.987429	validation_1-auc:0.977589
[13]	validation_0-auc:0.987802	validation_1-auc:0.97805
[14]	validation_0-auc:0.987676	validation_1-auc:0.97783
[

[141]	validation_0-auc:0.996165	validation_1-auc:0.983001
[142]	validation_0-auc:0.996188	validation_1-auc:0.98301
[143]	validation_0-auc:0.996222	validation_1-auc:0.983023
[144]	validation_0-auc:0.996248	validation_1-auc:0.983025
[145]	validation_0-auc:0.996272	validation_1-auc:0.983024
[146]	validation_0-auc:0.996295	validation_1-auc:0.983032
[147]	validation_0-auc:0.996321	validation_1-auc:0.983055
[148]	validation_0-auc:0.996348	validation_1-auc:0.983055
[149]	validation_0-auc:0.996372	validation_1-auc:0.983058
[150]	validation_0-auc:0.996397	validation_1-auc:0.983061
[151]	validation_0-auc:0.996423	validation_1-auc:0.983066
[152]	validation_0-auc:0.996443	validation_1-auc:0.983064
[153]	validation_0-auc:0.996462	validation_1-auc:0.983064
[154]	validation_0-auc:0.996482	validation_1-auc:0.983078
[155]	validation_0-auc:0.996506	validation_1-auc:0.983087
[156]	validation_0-auc:0.996527	validation_1-auc:0.983085
[157]	validation_0-auc:0.996545	validation_1-auc:0.983097
[158]	validatio

[283]	validation_0-auc:0.99842	validation_1-auc:0.983419
[284]	validation_0-auc:0.998438	validation_1-auc:0.983423
[285]	validation_0-auc:0.998449	validation_1-auc:0.983421
[286]	validation_0-auc:0.998475	validation_1-auc:0.983417
[287]	validation_0-auc:0.998482	validation_1-auc:0.983417
[288]	validation_0-auc:0.998487	validation_1-auc:0.983415
[289]	validation_0-auc:0.998512	validation_1-auc:0.983407
[290]	validation_0-auc:0.998531	validation_1-auc:0.983406
[291]	validation_0-auc:0.998541	validation_1-auc:0.983406
[292]	validation_0-auc:0.998559	validation_1-auc:0.983421
[293]	validation_0-auc:0.99856	validation_1-auc:0.98342
[294]	validation_0-auc:0.998586	validation_1-auc:0.983427
[295]	validation_0-auc:0.998593	validation_1-auc:0.983427
[296]	validation_0-auc:0.998609	validation_1-auc:0.983425
[297]	validation_0-auc:0.998623	validation_1-auc:0.983426
[298]	validation_0-auc:0.998635	validation_1-auc:0.983427
[299]	validation_0-auc:0.998647	validation_1-auc:0.983429
[300]	validation_

[425]	validation_0-auc:0.999534	validation_1-auc:0.983441
[426]	validation_0-auc:0.999536	validation_1-auc:0.983439
[427]	validation_0-auc:0.999541	validation_1-auc:0.983442
[428]	validation_0-auc:0.999548	validation_1-auc:0.983441
[429]	validation_0-auc:0.99955	validation_1-auc:0.983442
[430]	validation_0-auc:0.999556	validation_1-auc:0.983441
[431]	validation_0-auc:0.999557	validation_1-auc:0.983444
[432]	validation_0-auc:0.999564	validation_1-auc:0.983445
[433]	validation_0-auc:0.999567	validation_1-auc:0.983447
[434]	validation_0-auc:0.999571	validation_1-auc:0.983442
[435]	validation_0-auc:0.999575	validation_1-auc:0.983439
[436]	validation_0-auc:0.999581	validation_1-auc:0.983442
[437]	validation_0-auc:0.999586	validation_1-auc:0.983445
[438]	validation_0-auc:0.999587	validation_1-auc:0.983446
[439]	validation_0-auc:0.999592	validation_1-auc:0.983443
[440]	validation_0-auc:0.999596	validation_1-auc:0.983446
[441]	validation_0-auc:0.999598	validation_1-auc:0.983446
[442]	validatio

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.03, max_delta_step=0,
       max_depth=10, min_child_weight=1, missing=None, n_estimators=20000,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=420, silent=True, subsample=0.8)

In [19]:
pred_y = models.predict(test_x)
(pred_y==test_y).sum()/test_y.count()

/home/paperspace/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9417819714317681

In [179]:
pred_y = pd.DataFrame(model.predict(test_x))
pred_y = pred_y[0].between(3750901.5068, 3770901.5068) \
    & pred_y[1].between((-19268905.6133), -19208905.6133)
real_y = (test_y['x_exit_19'].between(3750901.5068, 3770901.5068) \
    & test_y['y_exit_19'].between((-19268905.6133), -19208905.6133)).reset_index(drop=True)
(pred_y==real_y).sum()/real_y.count()

In [26]:
models2 = Sequential()
models2.add(Dense(500, input_shape=(313,)))
models2.add(Activation('sigmoid'))
models2.add(Dense(500))
models2.add(Activation('sigmoid'))
# model.add(Dense(500))
# model.add(Activation('sigmoid'))
models2.add(Dense(1, activation='sigmoid'))
# model.add(Dense(2))
models2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
models2.fit(X,Y, epochs=12, batch_size=200)

Epoch 1/12
134063/134063 [==============================] - 4s 31us/step - loss: 0.4117 - acc: 0.8160
Epoch 2/12
134063/134063 [==============================] - 4s 29us/step - loss: 0.2577 - acc: 0.8965
Epoch 3/12
134063/134063 [==============================] - 4s 29us/step - loss: 0.2071 - acc: 0.9159
Epoch 4/12
134063/134063 [==============================] - 4s 29us/step - loss: 0.1898 - acc: 0.9226
Epoch 5/12
134063/134063 [==============================] - 4s 29us/step - loss: 0.1820 - acc: 0.9255
Epoch 6/12
134063/134063 [==============================] - 4s 29us/step - loss: 0.1751 - acc: 0.9284
Epoch 7/12
134063/134063 [==============================] - 4s 29us/step - loss: 0.1712 - acc: 0.9296
Epoch 8/12
134063/134063 [==============================] - 4s 29us/step - loss: 0.1669 - acc: 0.9318
Epoch 9/12
134063/134063 [==============================] - 4s 29us/step - loss: 0.1628 - acc: 0.9337
Epoch 10/12
134063/134063 [==============================] - 4s 29us/step - loss: 

In [35]:
models2 = xgb.XGBClassifier(max_depth=10, learning_rate=0.03, n_estimators=351, n_jobs=-1,
                           subsample=0.8, colsample_bytree=0.8, seed=420)

In [36]:
models2.fit(X, Y, eval_metric='auc', eval_set=[(X, Y)])


[0]	validation_0-auc:0.983069
[1]	validation_0-auc:0.984035
[2]	validation_0-auc:0.981881
[3]	validation_0-auc:0.983392
[4]	validation_0-auc:0.984867
[5]	validation_0-auc:0.984465
[6]	validation_0-auc:0.985399
[7]	validation_0-auc:0.9855
[8]	validation_0-auc:0.985857
[9]	validation_0-auc:0.9855
[10]	validation_0-auc:0.985394
[11]	validation_0-auc:0.985995
[12]	validation_0-auc:0.986464
[13]	validation_0-auc:0.98631
[14]	validation_0-auc:0.986126
[15]	validation_0-auc:0.986536
[16]	validation_0-auc:0.986954
[17]	validation_0-auc:0.986793
[18]	validation_0-auc:0.987156
[19]	validation_0-auc:0.98734
[20]	validation_0-auc:0.987596
[21]	validation_0-auc:0.987821
[22]	validation_0-auc:0.987795
[23]	validation_0-auc:0.987778
[24]	validation_0-auc:0.987659
[25]	validation_0-auc:0.987551
[26]	validation_0-auc:0.987618
[27]	validation_0-auc:0.987927
[28]	validation_0-auc:0.987869
[29]	validation_0-auc:0.988138
[30]	validation_0-auc:0.988379
[31]	validation_0-auc:0.988415
[32]	validation_0-auc:0.

[261]	validation_0-auc:0.997496
[262]	validation_0-auc:0.997507
[263]	validation_0-auc:0.997541
[264]	validation_0-auc:0.997548
[265]	validation_0-auc:0.997551
[266]	validation_0-auc:0.997562
[267]	validation_0-auc:0.997571
[268]	validation_0-auc:0.997593
[269]	validation_0-auc:0.9976
[270]	validation_0-auc:0.99761
[271]	validation_0-auc:0.997628
[272]	validation_0-auc:0.997639
[273]	validation_0-auc:0.99765
[274]	validation_0-auc:0.997663
[275]	validation_0-auc:0.997702
[276]	validation_0-auc:0.997713
[277]	validation_0-auc:0.997734
[278]	validation_0-auc:0.997748
[279]	validation_0-auc:0.997766
[280]	validation_0-auc:0.997774
[281]	validation_0-auc:0.997787
[282]	validation_0-auc:0.997794
[283]	validation_0-auc:0.997807
[284]	validation_0-auc:0.997825
[285]	validation_0-auc:0.99784
[286]	validation_0-auc:0.997849
[287]	validation_0-auc:0.997873
[288]	validation_0-auc:0.997898
[289]	validation_0-auc:0.997911
[290]	validation_0-auc:0.997918
[291]	validation_0-auc:0.997926
[292]	validat

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.03, max_delta_step=0,
       max_depth=10, min_child_weight=1, missing=None, n_estimators=351,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=420, silent=True, subsample=0.8)

In [65]:
X[:5]

,vmax_0,vmin_0,vmean_0,x_entry_0,y_entry_0,x_exit_0,y_exit_0,duration_0,entry_hour_0,entry_minute_0,...,exit_second_18,x_entry_19,y_entry_19,duration_19,entry_hour_19,entry_minute_19,entry_second_19,exit_hour_19,exit_minute_19,exit_second_19
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0,3.744945e+06,-1.928183e+07,962,15,2,31,15,18,33
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.0,3.749088e+06,-1.926605e+07,1756,15,0,32,15,29,48
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,3.758738e+06,-1.937594e+07,2716,14,34,35,15,19,51
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,3.767866e+06,-1.917797e+07,0,15,28,54,15,28,54
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,54.0,3.747641e+06,-1.922695e+07,0,15,8,5,15,8,5


# 3. Make Prediction

In [27]:
test = pd.read_csv("data_test.csv")

In [28]:
test['time_entry'] = pd.to_datetime(test['time_entry'], format='%H:%M:%S')
test['time_exit']  = pd.to_datetime(test['time_exit'], format='%H:%M:%S')

In [29]:
test['duration'] = (test['time_exit'] - test['time_entry']).dt.total_seconds().astype(int)

In [30]:
test['entry_hour'] = test['time_entry'].dt.hour
test['entry_minute'] = test['time_entry'].dt.minute
test['entry_second'] = test['time_entry'].dt.second
test['exit_hour'] = test['time_exit'].dt.hour
test['exit_minute'] = test['time_exit'].dt.minute
test['exit_second'] = test['time_exit'].dt.second

In [31]:
tgroups = test.groupby("hash")

In [46]:
tdependence = []
pbar = tqdm(total=len(tgroups))
for name, g in tgroups:
    pbar.update(1)
    available = [g.iloc[i] for i in range(len(g))]
    nanpaddin = [pd.Series([np.nan]*(20*len(test[:5].columns)-sum([len(x) for x in available])))]
    tdependence.append(pd.concat(nanpaddin+available, axis=0, ignore_index=True))    
pbar.close()

100%|██████████| 33515/33515 [01:08<00:00, 490.47it/s]


In [47]:
test2 = pd.concat(tdependence, axis=1).T

In [48]:
test2.columns = [x + "_" + str(y) for y in range(20) for x in test.columns]

In [49]:
test2[:5]

,Unnamed: 0_0,hash_0,trajectory_id_0,time_entry_0,time_exit_0,vmax_0,vmin_0,vmean_0,x_entry_0,y_entry_0,...,y_entry_19,x_exit_19,y_exit_19,duration_19,entry_hour_19,entry_minute_19,entry_second_19,exit_hour_19,exit_minute_19,exit_second_19
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.91449e+07,NaN,NaN,420,15,3,32,15,10,32
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.93414e+07,NaN,NaN,107,15,29,9,15,30,56
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.92386e+07,NaN,NaN,0,15,26,8,15,26,8
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.9355e+07,NaN,NaN,407,15,35,18,15,42,5
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.91701e+07,NaN,NaN,667,14,54,7,15,5,14


In [50]:
test2.to_csv('data_test_concat.csv')

In [32]:
test2 = pd.read_csv("data_test_concat.csv", index_col=False)

/home/paperspace/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
test2.shape

(33515, 381)

In [34]:
tname = test2['trajectory_id_19']

In [35]:
tX = test2.drop([x+str(y) for y in range(20) for x in ['Unnamed: 0_', 'hash_', 'trajectory_id_', 'time_entry_', 'time_exit_']] \
    + ['x_exit_19', 'y_exit_19','vmax_19', 'vmin_19', 'vmean_19'] + ['Unnamed: 0'], axis=1)
# X = X.fillna(0)

In [36]:
# extrapolation
for i in range(18,-1,-1):
    cur = str(i)
    nex = str(i+1)
    nan = tX['x_entry_'+cur].isnull()
    for x,y in zip(['x_entry_'+cur, 'y_entry_'+cur, 'entry_hour_'+cur, 'entry_minute_'+cur, 'entry_second_'+cur],
                   ['x_entry_'+nex, 'y_entry_'+nex, 'entry_hour_'+nex, 'entry_minute_'+nex, 'entry_second_'+nex]):
        tX.iloc[np.where(nan)[0], tX.columns.get_loc(x)] = tX[y][nan]
    for x,y in zip(['x_exit_'+cur, 'y_exit_'+cur, 'exit_hour_'+cur, 'exit_minute_'+cur, 'exit_second_'+cur],
                   ['x_entry_'+nex, 'y_entry_'+nex, 'entry_hour_'+nex, 'entry_minute_'+nex, 'entry_second_'+nex]):
        tX.iloc[np.where(nan)[0], tX.columns.get_loc(x)] = tX[y][nan]

In [37]:
for i in range(19):
    tX['distance_'+str(i)] = ((tX['x_entry_'+str(i)]-tX['x_exit_'+str(i)]).pow(2)+(tX['y_entry_'+str(i)]-tX['y_exit_'+str(i)]).pow(2)).pow(0.5)
    tX['speed_'+str(i)] = tX['distance_'+str(i)]/tX['duration_'+str(i)]
tX = tX.replace([np.inf, -np.inf], np.nan)
tX = tX.fillna(0)

In [38]:
tX = scaler.transform(tX)

In [39]:
(X.shape, tX.shape)

((134063, 313), (33515, 313))

In [40]:
tX[:5]

array([[-0.01030431, -0.00965435, -0.01013875, ...,  0.07376936,
         0.65312728,  0.04161556],
       [-0.01030431, -0.00965435, -0.01013875, ..., -0.03684396,
        -0.0306996 ,  0.05730477],
       [-0.01030431, -0.00965435, -0.01013875, ...,  0.11200405,
        -0.35368926, -0.07931455],
       [-0.01030431, -0.00965435, -0.01013875, ..., -0.07865168,
        -0.35368926, -0.07931455],
       [-0.01030431, -0.00965435, -0.01013875, ..., -0.07708621,
         0.16464553, -0.0134529 ]])

In [61]:
pred_y = models2.predict(tX)

In [74]:
pred_y = models2.predict_classes(tX)
pred_y = pred_y.reshape((1,-1))[0]

In [75]:
print(pred_y.sum(), pred_y.shape)

8434 (33515,)


In [76]:
output = pd.DataFrame({'id': tname, 'target': pred_y.astype(int)})
output.to_csv('submission8.csv', index=False, columns=['id', 'target'])

In [77]:
pred_y.max()

1